# model.data 
본 노트에서는 `torch.utils.data.Dataset` class를 subclassing하여 custom Dataset인 `Corpus` class를 구현하고, 이와 함께`torch.utils.data.DataLoader` class를 이용하여 딥러닝 모형을 위한 data pipeline을 구현해보도록 합니다. 이 때, `build_vocab.ipynb` 노트에서 활용했었던 `Vocab` class, `Tokenizer` class, `PadSequence` class를 이용하여 전처리 과정을 data pipeline에 통합하도록 합니다. 구현한 `Corpus` class는 모듈화시 data pipeline에 관련된 script에 존재합니다.

### Setup

In [1]:
import pickle
import torch 
import pandas as pd
from pathlib import Path
from typing import Callable, List, Tuple
from pprint import pprint
from mecab import MeCab
from torch.utils.data import Dataset, DataLoader
from model.utils import Vocab, Tokenizer, PadSequence

### `Corpus` class
`Corpus` class는 PyTorch official tutorial 중 하나인 [Data Loading and Processing Tutorial](https://pytorch.org/tutorials/beginner/data_loading_tutorial.html#data-loading-and-processing-tutorial)을 따라하면 쉽게 구현할 수 있습니다. 마찬가지로 `__init__`, `__len__`, `__getitem__`을 method overriding으로 구현하도록 합니다. `__getitem__`을 구현할 때, 필요한 전처리를 안쪽에 구현하는 것이 포인트입니다. `Corpus` class가 parameter로 전처리에 관련된 함수를 전달받아서 이를 `__getitem__`에서 활용하도록 구현합니다.

In [2]:
class Corpus(Dataset):
    """Corpus class"""
    def __init__(self, filepath: str, transform_fn: Callable[[str], List[int]]) -> None:
        """Instantiating Corpus class

        Args:
            filepath (str): filepath
            transform_fn (Callable): a function that can act as a transformer
        """
        self._corpus = pd.read_csv(filepath, sep='\t').loc[:, ['document', 'label']]
        self._transform = transform_fn

    def __len__(self) -> int:
        return len(self._corpus)

    def __getitem__(self, idx: int) -> Tuple[torch.Tensor, torch.Tensor]:
        tokens2indices = torch.tensor(self._transform(self._corpus.iloc[idx]['document']))
        label = torch.tensor(self._corpus.iloc[idx]['label'])
        return tokens2indices, label

위와 같이 구현하면 `Corpus` class의 `transform_fn` argument에 전처리에 관련된 함수를 parameter로 전달함으로써 매우 쉽게 data pipeline의 일부분을 구현할 수 있습니다. 이를 위해서 아래의 단계를 진행합니다.

#### `build_vocab.ipynb` 노트에서 생성했던 `Vocab` class의 instance를 `split_fn`으로 활용했던 `Mecab` instance의 `morphs` 함수를 불러오고, `PadSequence`의 instance를 생성합니다. 이때 padding의 길이는 `eda.ipynb` 노트에서 탐색했던 결과로 padding합니다.

In [3]:
data_dir = Path.cwd() / 'data'
list_of_dataset = list(data_dir.iterdir())

In [4]:
pprint(list_of_dataset)
with open(list_of_dataset[-1], mode='rb') as io:
    vocab = pickle.load(io)

[PosixPath('/root/Documents/archive/strnlp/exercise/data/.DS_Store'),
 PosixPath('/root/Documents/archive/strnlp/exercise/data/morphs_eojeol.pkl'),
 PosixPath('/root/Documents/archive/strnlp/exercise/data/train.txt'),
 PosixPath('/root/Documents/archive/strnlp/exercise/data/morphs_vec.pkl'),
 PosixPath('/root/Documents/archive/strnlp/exercise/data/validation.txt'),
 PosixPath('/root/Documents/archive/strnlp/exercise/data/tokenizer.pkl'),
 PosixPath('/root/Documents/archive/strnlp/exercise/data/test.txt'),
 PosixPath('/root/Documents/archive/strnlp/exercise/data/vocab.pkl')]


In [5]:
# 문장을 어절기준으로 보는 split_fn을 작성
def split_eojeol(s: str) -> List[str]:
    return s.split(' ')

In [6]:
pad_sequence = PadSequence(length=32, pad_val=vocab.to_indices(vocab.padding_token))

#### `Tokenizer` class의  instance를 생성할 때, `vocab`, `split_morphs`, `pad_sequence`를 parameter로 전달하여 instance를 생성합니다.

In [7]:
tokenizer = Tokenizer(vocab=vocab, split_fn=split_eojeol, pad_fn=pad_sequence)

#### `Corpus` class의 instance를 생성할 때, `transform_fn` argument에 `tokenizer.split_and_transform`  멤버함수를 parameter로 전달합니다.

In [8]:
tr_corpus = Corpus(list_of_dataset[2], transform_fn=tokenizer.split_and_transform)

In [9]:
tr_corpus._corpus.head()

,document,label
0,애들 욕하지마라 지들은 뭐 그렇게 잘났나? 솔까 거기 나오는 귀여운 애들이 당신들보...,1
1,여전히 반복되고 있는 80년대 한국 멜로 영화의 유치함.,0
2,쉐임리스 스티브와 피오나가 손오공 부르마로 ㅋㅋㅋ,0
3,0점은 없나요?...,0
4,제발 시즌2 ㅜㅜ,1


### `DataLoader` class
`torch.utils.data.DataLoader` class의 instance를 생성할 때, `dataset` argument에 위에서 생성한 `Corpus` class의 instance를 전달합니다.

In [10]:
# example
tr_dl = DataLoader(tr_corpus, shuffle=False, batch_size=128, num_workers=4)

In [11]:
# # epoch=2, batch=128로 DataLoader (tr_dl)를 순회하는 for문을 작성해보세요
# epochs = 2

In [ ]:
for epoch in range(epochs):
    
    for mb in tr_dl:
        x_mb, y_mb = mb